In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/smltest2/sample_submission.csv
/kaggle/input/smltest2/train.csv
/kaggle/input/smltest2/test.csv


In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
import os


In [24]:
# Load datasets
train_df = pd.read_csv("/kaggle/input/smltest2/train.csv")
test_df = pd.read_csv("/kaggle/input/smltest2/test.csv")
sample_submission = pd.read_csv("/kaggle/input/smltest2/sample_submission.csv")


In [25]:
# Clean TotalCharges
train_df['TotalCharges'] = pd.to_numeric(train_df['TotalCharges'], errors='coerce')
test_df['TotalCharges'] = pd.to_numeric(test_df['TotalCharges'], errors='coerce')
train_df['TotalCharges'] = train_df['TotalCharges'].fillna(train_df['TotalCharges'].median())
test_df['TotalCharges'] = test_df['TotalCharges'].fillna(test_df['TotalCharges'].median())


In [26]:
# Feature Engineering (keep only best features)
for df in [train_df, test_df]:
    df['AvgChargePerMonth'] = df['TotalCharges'] / df['tenure']
    df['AvgChargePerMonth'] = df['AvgChargePerMonth'].replace([np.inf, -np.inf], 0)
    df['AvgChargePerMonth'] = df['AvgChargePerMonth'].fillna(0)
    df['HasInternet'] = (df['InternetService'] != 'No').astype(int)
    df['IsEngaged'] = ((df['StreamingTV'] != 'No') | (df['StreamingMovies'] != 'No')).astype(int)
            

In [27]:
# Prepare features and target
X = train_df.drop(columns=['ID', 'customerID', 'Churn'])
y = train_df['Churn']
X_test_final = test_df.drop(columns=['ID', 'customerID'])


In [28]:
# One-hot encoding
X = pd.get_dummies(X, drop_first=True)
X_test_final = pd.get_dummies(X_test_final, drop_first=True)
X_test_final = X_test_final.reindex(columns=X.columns, fill_value=0)

In [29]:
# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test_final)

In [30]:
# Train/Validation Split
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [31]:
# Handle class imbalance
pos_weight = (y_train == 0).sum() / (y_train == 1).sum()


In [32]:
# Train XGBoost model (stable params)
model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    scale_pos_weight=pos_weight,
    max_depth=4,
    learning_rate=0.05,
    n_estimators=100,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=1,
    random_state=42
)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=1, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [33]:
# Threshold optimization
y_val_probs = model.predict_proba(X_val)[:, 1]
best_thresh, best_f1 = 0.5, 0
for t in np.arange(0.3, 0.7, 0.01):
    preds = (y_val_probs >= t).astype(int)
    f1 = f1_score(y_val, preds)
    if f1 > best_f1:
        best_f1, best_thresh = f1, t
print(f"Best F1 Score: {best_f1:.4f} | Best Threshold: {best_thresh:.3f}")

Best F1 Score: 0.6376 | Best Threshold: 0.540


In [34]:
# Retrain on full data
final_model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    scale_pos_weight=pos_weight,
    max_depth=4,
    learning_rate=0.05,
    n_estimators=100,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=1,
    random_state=42
)
final_model.fit(X_scaled, y)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=1, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [35]:
# Final predictions with best threshold
final_probs = final_model.predict_proba(X_test_scaled)[:, 1]
final_preds = (final_probs >= best_thresh).astype(int)

In [36]:
# Save to submission file
sample_submission['Churn'] = final_preds
sample_submission.to_csv("/kaggle/working/submission.csv", index=False)

